Import libraries

In [ ]:
import sys
from silx.io.dictdump import h5todict

sys.path.append("/home/zanardi/Codes/ML/RONEK/ronek/")

from ronek import env
env.set(
  device="cpu",
  device_idx=0,
  nb_threads=4,
  floatx="float64"
)

from ronek.roms import CoarseGrainingM0

Set inputs

In [ ]:
T = 1e4
nb_bins = 45
dist = "boltzmann" # uniform/boltzmann
kin_file = "/home/zanardi/Workspace/AirDatabase/HDF5/QCT/Orig/O3_UMN.hdf5"
mol_file = "./../species/molecule.json"

Load the kinteic database

In [ ]:
kin_fom = h5todict(kin_file)

Build the zeroth-order coarse-grained ROM

In [ ]:
cg = CoarseGrainingM0(mol_file)
cg(T=T, nb_bins=nb_bins)

Encode-decode rates

In [ ]:
Tstr = str(int(T))
rates = kin_fom[f"T_{Tstr}_{Tstr}"]["Rates"]

kin_rom = {}
# Inelastic rates
# ------------------
name = "Inel"
krom = cg.phi.T @ rates[name] @ cg.psi
kin_rom[name] = {
  # > Encoded rates
  "rom": krom,
  # > Encoded-decoded rates
  "fom": cg.psi @ krom @ cg.psi.T if (dist == "uniform") else cg.psi @ krom @ cg.phi.T
}

# Dissociation rates
# ------------------
name = "Diss_Corr"
krom = cg.phi.T @ rates[name]
kin_rom[name] = {
  # > Encoded rates
  "rom": krom,
  # > Encoded-decoded rates
  "fom": cg.psi @ krom
}